In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.chdir(r'D:\小鸡理财\百度云同步盘\小鸡理财\周报')

In [3]:
with pd.ExcelFile('回款（2018.7.23-7.29）.xlsx') as xlsx:
    df_org = pd.read_excel(xlsx,'经典')
    df_cg = pd.read_excel(xlsx,'存管')

In [ ]:
cf = pd.read_excel(

In [17]:
num = df_org.shape[0] + df_cg.shape[0]

num

43632

In [12]:
df_org.rename(columns = {'标类型':'投资期限','总投资额':'发放金额','还款时间':'发放时间'},inplace=True)

df_cg_need = df_cg[['投资期限','本次发放金额','预计本次发放时间','身份证']].rename(columns={'本次发放金额':'发放金额','预计本次发放时间':'发放时间'})

df_org.head()

,投资期限,发放金额,发放时间,身份证
0,180,32000.0,2018-08-16,330227196610152023
1,180,100.0,2018-08-16,371324198511244911
2,180,200.0,2018-08-16,331004198602141812
3,180,100.0,2018-08-16,330702198109046058
4,180,10000.0,2018-08-16,140429199006058526


In [21]:
df_org_cg = df_org.append(df_cg_need,ignore_index=True)

df_org_cg.shape

df_org_cg.head()

df_org_cg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43632 entries, 0 to 43631
Data columns (total 4 columns):
投资期限    43632 non-null int64
发放金额    43632 non-null float64
发放时间    43632 non-null object
身份证     43632 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 1.3+ MB


In [24]:
df_org_cg['发放时间'] = pd.to_datetime(df_org_cg['发放时间'])

df_org_cg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43632 entries, 0 to 43631
Data columns (total 4 columns):
投资期限    43632 non-null int64
发放金额    43632 non-null float64
发放时间    43632 non-null datetime64[ns]
身份证     43632 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.3+ MB


In [57]:
# table = pd.pivot_table(df,values=['City'],index=['Province'],columns=['City'],aggfunc=np.size)

table = pd.pivot_table(df_org_cg,values = ['发放金额'],index = ['发放时间'],columns = ['投资期限'],aggfunc = np.sum)

table.head()

发放金额                          
投资期限        30  90         180        360
发放时间                                     
2018-07-01  NaN NaN  961355.58  231772.91
2018-07-02  NaN NaN  276787.80  121479.34
2018-07-03  NaN NaN  133984.96  286190.21
2018-07-04  NaN NaN  484443.95  156688.63
2018-07-05  NaN NaN  307238.64  110354.34

In [40]:
res = table.truncate(before='20180723',after='20180729')

res

发放金额                                   
投资期限              30          90         180         360
发放时间                                                    
2018-07-23  211002.87  1554997.85  476463.49   805206.04
2018-07-24        NaN   811996.15  365053.08  1040087.11
2018-07-25        NaN   385424.93  570119.01   895659.42
2018-07-26        NaN  1408103.38  432557.57   890677.68
2018-07-27        NaN   207502.27  466575.86   686914.69
2018-07-28        NaN   804442.14  485753.74   784221.22
2018-07-29        NaN  1259740.61  444998.52   728728.18

In [41]:
res['总和'] = res.transform(np.sum,axis=1)

res

发放金额                                             总和
投资期限               30          90        180         360            
发放时间                                                                
2018-07-23  211002.87  1554997.85  476463.49   805206.04  3047670.25
2018-07-24        NaN   811996.15  365053.08  1040087.11  2217136.34
2018-07-25        NaN   385424.93  570119.01   895659.42  1851203.36
2018-07-26        NaN  1408103.38  432557.57   890677.68  2731338.63
2018-07-27        NaN   207502.27  466575.86   686914.69  1360992.82
2018-07-28        NaN   804442.14  485753.74   784221.22  2074417.10
2018-07-29        NaN  1259740.61  444998.52   728728.18  2433467.31

In [42]:
res.to_excel('01.xlsx')
print('ok')

ok


In [59]:
pp = df_org_cg.groupby(['发放时间','身份证'],as_index=False).agg({'发放金额':np.sum})

pp.head()

,发放时间,身份证,发放金额
0,2018-07-01,110102198305263080,100.62
1,2018-07-01,110221196906285029,144.57
2,2018-07-01,120104195802052525,605.75
3,2018-07-01,130429198212091662,67.50
4,2018-07-01,130604197802220339,3032.50


In [65]:
pp['rank'] = pp.groupby(['发放时间'])['发放金额'].rank(ascending=0,method='min')

tt = pp.sort_values(by=['发放时间','rank'], axis=0, ascending=[True,True])

tt.head()

,发放时间,身份证,发放金额,rank
208,2018-07-01,330902197204065229,111341.69,1.0
71,2018-07-01,330203196610243047,101194.30,2.0
88,2018-07-01,330205196406262227,50545.90,3.0
163,2018-07-01,330227196909274922,50545.90,3.0
82,2018-07-01,330204197305066014,50541.70,5.0


In [84]:
kk = tt[(tt['rank'] > 0) & (tt['rank'] < 11)]

kk.set_index('发放时间',inplace=True)

kk.info()

res = kk.truncate(before='20180723',after='20180729')

res

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2341 entries, 2018-07-01 to 2019-02-13
Data columns (total 3 columns):
身份证     2341 non-null object
发放金额    2341 non-null float64
rank    2341 non-null float64
dtypes: float64(2), object(1)
memory usage: 73.2+ KB


,身份证,发放金额,rank
发放时间,,,
2018-07-23,330281198810016322,129164.72,1.0
2018-07-23,330211195912040021,101698.79,2.0
2018-07-23,330227198703243441,101041.68,3.0
2018-07-23,33042219700820332x,98030.54,4.0
2018-07-23,330204197804012029,96155.27,5.0
2018-07-23,330902195011120624,70816.74,6.0
2018-07-23,330227196909274922,70637.54,7.0
2018-07-23,330205196804050027,69931.29,8.0
2018-07-23,330205198111203368,61514.58,9.0
